Example 3.4 pg 234

Working this example to try to understand how to do p4 


In [1]:
from sympy import symbols, diff,exp
import numpy as np
import math
from tabulate import tabulate

The first part is relatively straight forward. Calculating the interpolated values based on the provided divide difference table for the polynomials of degrees 1,2,3. Calculating the actual error by evaluating the provided equation at x = 1.75 and comparing it to the interpolated results for the polynomials of varying degrees. 

The second part is convoluted. The only real, and somewhat foreboding, hint from the authors' is "(MATLAB helped in finding the derivatives and evaluating the maximum and minimum values witin the intervals.)".

This notebook is focused on determining the last four columns of the following table

In [2]:
iv = np.array([1.25668, 1.28520, 1.28661])
ae = np.array([0.01996, -0.00856, -0.00947])
fn1_max = np.array([-0.3679, -0.8661, 1.1398])
fn1_min = np.array([0.0594, 0.1249, -0.0359])
ub = np.array([0.0299, 0.0059, 0.0014])
lb = np.array([-0.00483, -0.0408, -0.4039])

tb_h = ['Interpolated value', 'Actual error', 'f^(n+1) max', 'f^(n+1) min', 'upper bound', 'lower bound']
tb = np.column_stack([iv, ae, fn1_max, fn1_min, ub, lb])
print(tabulate(tb, headers=tb_h))

  Interpolated value    Actual error    f^(n+1) max    f^(n+1) min    upper bound    lower bound
--------------------  --------------  -------------  -------------  -------------  -------------
             1.25668         0.01996        -0.3679         0.0594         0.0299       -0.00483
             1.2852         -0.00856        -0.8661         0.1249         0.0059       -0.0408
             1.28661        -0.00947         1.1398        -0.0359         0.0014       -0.4039


In the following cell are the calculations for the 2nd, 3rd, and 4th derivatives of the given function. This will allow for the calculation of f^(n+1)(e) where e will be taken as {x0, x, ..., xn}. The results from this evaluation will produce the maximum and minimum values of the (n+1)st derivatives which are necessary in determining the error bounds. 

In the following calculations:
x0 = 1.1,
x = 1.75,
x1 = 2.0,
x2 = 3.5,
x3 = 5.0,
x4 = 7.0

In [3]:
x = symbols("x")
f = x**2*exp(-x/2)
fd2 = diff(f, x, 2)
fd3 = diff(f, x, 3)
fd4 = diff(f, x, 4)

print(fd2)
print(fd3)
print(fd4)

def eval(xi):
    return fd2.subs(x,xi),fd3.subs(x,xi),fd4.subs(x,xi)
    
arr = np.empty(shape=(5,3), dtype=float)
c = 0
xl = [1.1, 1.75, 2., 3.5, 5., 7.1]
for xi in xl:
    arr[c:] = eval(xi)
    c+=1
    
tb_h1 = ['f^(2)', 'f^(3)', 'f^(4)']
print(tabulate(arr, tb_h1))

(x**2/4 - 2*x + 2)*exp(-x/2)
(-x**2/8 + 3*x/2 - 3)*exp(-x/2)
(x**2/16 - x + 3)*exp(-x/2)
     f^(2)      f^(3)       f^(4)
----------  ---------  ----------
 0.0591374  -0.866146   1.13984
-0.306133   -0.315903   0.600868
-0.367879   -0.18394    0.459849
-0.336687    0.1249     0.0461587
-0.143649    0.112867  -0.0359122


In the following cell is the filtering and selection of the maximum and minimum values of the 2nd, 3rd and 4th derivatives evaluated at {x0, x, ..., xn}. 

The first two results consider the values determined from the 2nd derivative evaluated at {x0, x, x1} because these are the values used in calculating the interpolated value and where the error needs to be valued for this a polynomial of degree 1.

The second two results consider the values determined from the 3rd derivative evaluated at {x0, x, x1, x2} for a 2nd degree polynomial interpolation.

The last two results consider the values determine from the 4th derivative evaluated at {x0, x, x1, x2, x3} for a 3rd degree polynomial interpolation.

In [9]:
d = np.array([
    [np.amax(arr[0:3,0], 0), np.amin(arr[0:3,0], 0)],
    [np.amax(arr[0:4,1], 0), np.amin(arr[0:4,1], 0)],
    [np.amax(arr[0:5,2], 0), np.amin(arr[0:5,2], 0)]])

print(arr[0:3,0])
print(arr[0:4,1])
print(arr[0:5,2])
print(tabulate(d, headers=tb_h[2:4]))


[ 0.05913736 -0.30613305 -0.36787944]
[-0.8661459  -0.31590325 -0.18393972  0.12490002]
[ 1.13983647  0.60086752  0.4598493   0.0461587  -0.03591219]
  f^(n+1) max    f^(n+1) min
-------------  -------------
    0.0591374     -0.367879
    0.1249        -0.866146
    1.13984       -0.0359122


Finally to determine to the error bounds use the following equation with the values given above:

E(x) = (x-x0)(x-x1)...(x-xn)\*f^(n+1)(e)/(n+1)!

After E(x) is evaluated at each of the two values (max and min) for the derivative, the maximum result is the upper bound and minimum result is the lower bound. 

In [5]:
e1_a = (xl[1] - xl[0])*(xl[1] - xl[2])*d[0,0]/math.factorial(2)
e1_b = (xl[1] - xl[0])*(xl[1] - xl[2])*d[0,1]/math.factorial(2)
e2_a = (xl[1] - xl[0])*(xl[1] - xl[2])*(xl[1] - xl[3])*d[1,0]/math.factorial(3)
e2_b = (xl[1] - xl[0])*(xl[1] - xl[2])*(xl[1] - xl[3])*d[1,1]/math.factorial(3)
e3_a = (xl[1] - xl[0])*(xl[1] - xl[2])*(xl[1] - xl[3])*(xl[1] - xl[4])*d[2,0]/math.factorial(4)
e3_b = (xl[1] - xl[0])*(xl[1] - xl[2])*(xl[1] - xl[3])*(xl[1] - xl[4])*d[2,1]/math.factorial(4)

e = np.array([[max(e1_a, e1_b), min(e1_a,e1_b)],[max(e2_a,e2_b),min(e2_a,e2_b)],[max(e3_a, e3_b), min(e3_a, e3_b)]])

print(tabulate(e, headers=tb_h[4:6]))

  upper bound    lower bound
-------------  -------------
   0.0298902     -0.00480491
   0.00591974    -0.0410517
   0.00138295    -0.0438941
